In [ ]:
from io import open
import unicodedata
import string
import re
import pandas as pd
import numpy as np
import random
from itertools import product
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
drive.mount('/content/drive')

In [ ]:
import os
import sys
from google.colab import drive

### 2. Импорт и предобработка текста

####2.1.

In [ ]:
!wget https://www.manythings.org/anki/rus-eng.zip
!unzip rus-eng.zip

--2023-03-26 14:49:59--  https://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15374406 (15M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  14.66M  7.84MB/s    in 1.9s    

2023-03-26 14:50:02 (7.84 MB/s) - ‘rus-eng.zip’ saved [15374406/15374406]

Archive:  rus-eng.zip
  inflating: rus.txt                 
  inflating: _about.txt              


In [ ]:
!head rus.txt

Go.	Марш!	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1159202 (shanghainese)
Go.	Иди.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #5898247 (marafon)
Go.	Идите.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #5898250 (marafon)
Hi.	Здравствуйте.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #402127 (odexed)
Hi.	Привет!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #466968 (katjka)
Hi.	Хай.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #467233 (timsa)
Hi.	Здрасте.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #3803577 (marafon)
Hi.	Здоро́во!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #3854188 (marafon)
Hi.	Приветик!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #7234283 (marafon)
Run!	Беги!	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #1569978 (Biga)


In [ ]:
SOS_token = 0
EOS_token = 1
lang1 = 'rus'
lang2 = 'eng'

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Zа-яА-Я.!?]+", r" ", s)
    return s

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines    #data/%s-%s.txt
    # lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
    lines = open('rus.txt', encoding='utf-8').read().strip().split('\n')

    # Split every line into pairs and normalize
    # pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    pairs = [[normalizeString(s) for s in l.split('\t')[:-1]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang  = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

#### 2.__

In [ ]:
rl = readLangs('eng', 'rus', reverse=False)[2]

Reading lines...


In [ ]:
rl[2000:2005]

[['who s tom ?', 'кто том ?'],
 ['who s tom ?', 'кто такои том ?'],
 ['who s ill ?', 'кто болен ?'],
 ['who s she ?', 'кто она ?'],
 ['wonderful !', 'замечательно !']]

In [ ]:
df_temp = pd.DataFrame(rl, columns = ['eng', 'rus'])
df_temp['eng_words'] =df_temp['eng'].str.split().apply(len)
df_temp['rus_words'] =df_temp['rus'].str.split().apply(len)
df_temp['min_words'] =df_temp[['rus_words', 'eng_words']].min(axis=1)
df_temp.tail()

,eng,rus,eng_words,rus_words,min_words
464005,at a moment when our economy is growing our bu...,в тот момент когда наша экономика растет наши ...,43,36,36
464006,when i was younger i hated going to weddings ....,когда я была помоложе я ненавидела ходить на с...,49,47,47
464007,since there are usually multiple websites on a...,поскольку саитов посвященных какои либо теме к...,46,40,40
464008,if someone who doesn t know your background sa...,если кто то незнакомыи говорит что вы говорите...,49,39,39
464009,doubtless there exists in this world precisely...,несомненно для каждого мужчины в этом мире где...,102,83,83


In [ ]:
df_temp['min_w_gr']  =pd.cut(df_temp['min_words'],[-9,4,5,6,7,8,9,10,20,9999])
df_temp['min_w_gr'].value_counts()

(4, 5]        100569
(5, 6]         98344
(-9, 4]        90260
(6, 7]         75136
(7, 8]         46857
(8, 9]         25911
(10, 20]       13771
(9, 10]        13041
(20, 9999]       121
Name: min_w_gr, dtype: int64

In [ ]:
eng_prefixes = ("i am ", "i m ",
                "he is", "he s ",
                "she is", "she s",
                "you are", "you re ",
                "we are", "we re ",
                "they are", "they re ")

df_temp['prefi'] = [df_temp.loc[i,'eng'].startswith(eng_prefixes)
                                                    for i in df_temp.index]

df_temp['prefi'].value_counts()

False    431782
True      32228
Name: prefi, dtype: int64

In [ ]:
df_temp[df_temp['prefi']==True].min_words.value_counts()

4     7114
5     6667
6     5655
7     4323
3     3558
8     2530
9     1213
10     628
11     275
12     118
13      53
14      34
2       33
15      10
16      10
17       3
18       3
20       1
Name: min_words, dtype: int64

In [ ]:
df_temp[(df_temp['prefi']==True)&(df_temp['min_words']>3)&(df_temp['min_words']<20)].shape[0]

28636

In [ ]:
df_temp[(df_temp['min_words']>3)&(df_temp['min_words']<7)].shape[0]

267012

In [ ]:
df_temp[(df_temp['min_words']>3)&(df_temp['min_words']<20)].shape[0]

441677

In [ ]:
df_temp['delta_len'] = abs(df_temp['eng_words'] - df_temp['rus_words'])
df_temp['delta_len'].value_counts()

1     185494
2     111598
0      98563
3      45143
4      15904
5       5166
6       1554
7        407
8        105
9         35
10        20
11        13
12         5
13         2
19         1
Name: delta_len, dtype: int64

In [ ]:
 df_temp[(df_temp['delta_len']>10)&(df_temp['min_words']<20)].iloc[:,[0,1,2,3,-1]][2:11]

,eng,rus,eng_words,rus_words,delta_len
462900,i promise i ll pay you back the money that i o...,обещаю что верну тебе долг до конца месяца .,20,9,11
462902,i promise i ll pay you back the money that i o...,обещаю что верну вам долг до конца месяца .,20,9,11
462904,i promise that i ll pay you back the money i o...,обещаю что верну тебе долг до конца месяца .,20,9,11
462906,i promise that i ll pay you back the money i o...,обещаю что верну вам долг до конца месяца .,20,9,11
463250,i promise that i ll pay you back the money tha...,обещаю что верну тебе долг до конца месяца .,21,9,12
463252,i promise that i ll pay you back the money tha...,обещаю что верну вам долг до конца месяца .,21,9,12
463622,it had the head of a woman the body of a lion ...,у него была голова женщины тело льва крылья пт...,24,13,11
463722,i ve heard it said that it s harder to please ...,я слышал что женщине труднее угодить чем мужчи...,26,15,11
463723,i ve heard it said that it s harder to please ...,я слышал что женщину труднее удовлетворить чем...,26,15,11


#### 2.2.

In [ ]:
def filterPair(p, prefi, max_):
    if prefi == None:
        return  len(p[0].split(' ')) >3 and \
                len(p[1].split(' ')) >3 and \
                len(p[0].split(' ')) < max_ and \
                len(p[1].split(' ')) < max_
    else:
        return  len(p[0].split(' ')) >3 and \
                len(p[1].split(' ')) >3 and \
                len(p[0].split(' ')) < max_ and \
                len(p[1].split(' ')) < max_ and \
                p[0].startswith(prefi) #

def filterPairs(pairs, pref, max_l):
    return [pair for pair in pairs if filterPair(pair, pref, max_l)]

In [ ]:
def prepareData(lang1, lang2, pre, max_le, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs, pre, max_le)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair, input_l, output_l):
    input_tensor = tensorFromSentence(input_l, pair[0])
    target_tensor = tensorFromSentence(output_l, pair[1])
    return (input_tensor, target_tensor)

# **7 The Encoder** (прогоняет наши данные через модель)
-----------




In [ ]:
class EncoderRNN2(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers, rnn_type):
        super(EncoderRNN2, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = rnn_type(hidden_size, hidden_size, n_layers)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.rnn(output, hidden)
        return output, hidden

    def initHidden(self, n_layers):
        return torch.zeros(n_layers, 1, self.hidden_size, device=device)

#  **8 The Decoder**
-----------


In [ ]:
class DecoderRNN2(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers, rnn_type):
        super(DecoderRNN2, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.rnn       = rnn_type(hidden_size, hidden_size, n_layers)
        self.out       = nn.Linear(hidden_size, output_size)
        self.softmax   = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output         = self.embedding(input).view(1, 1, -1)
        output         = F.relu(output)
        output, hidden = self.rnn(output, hidden)
        output         = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self, n_layers):
        return torch.zeros(n_layers, 1, self.hidden_size, device=device)

#### 3.3. Train + Teacher forcing

In [ ]:
def train(input_tensor, target_tensor,
          encoder, decoder, encoder_optimizer, decoder_optimizer,
          criterion, max_length, rnn_ty, n_layers, t_f_ratio):

    if rnn_ty.__name__ == 'LSTM':
        encoder_hidden = (encoder.initHidden(n_layers), encoder.initHidden(n_layers))
    else:
        encoder_hidden = encoder.initHidden(n_layers)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
                                            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < t_f_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                                                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                                                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

#### 3.4. Train + iters

In [ ]:
def trainIters(pair_sentenses, encoder, decoder, learning_rate, n_iters,
               MAX_LENGTH, rnn_typ, nu_layers, teach_f_ratio, input_lan, output_lan,
               print_every=5000, plot_every=500):

    start               = time.time()
    print_loss_total_loc = 0
    print_loss_avg = 0

    encoder_optimizer = optim.Adam(encoder.parameters())
    decoder_optimizer = optim.Adam(decoder.parameters())

    training_pairs = [tensorsFromPair(random.choice(pair_sentenses),
                                      input_lan, output_lan)
                                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder,
                     encoder_optimizer, decoder_optimizer, criterion,
                     MAX_LENGTH, rnn_typ, nu_layers, teach_f_ratio)
        print_loss_total_loc += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total_loc / print_every
            print_loss_total_loc = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))


    return print_loss_avg

#### 3.5. Evaluate

In [ ]:
def evaluate(encoder, decoder, sentence, max_length, n_layers,
             Rnn, inp_lang, out_lang):

    with torch.no_grad():
        input_tensor = tensorFromSentence(inp_lang, sentence)
        input_length = input_tensor.size()[0]

        if Rnn.__name__ == 'LSTM':
            encoder_hidden = (encoder.initHidden(n_layers), encoder.initHidden(n_layers))
        else:
            encoder_hidden = encoder.initHidden(n_layers)

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(out_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
def evaluateRandomly(pair_sen, encoder, decoder, MAX_LENGTH,
                     _lauers, rnn_, in_lang, o_lang,  n=10):
    for i in range(n):
        pair = random.choice(pair_sen)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0], MAX_LENGTH,
                                 _lauers, rnn_, in_lang, o_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

#### 3.6. Вспомогательные функции

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

### 4. Обучение

In [ ]:
eng_prefixes = ("i am ", "i m ",
                "he is", "he s ",
                "she is", "she s",
                "you are", "you re ",
                "we are", "we re ",
                "they are", "they re "
                )

rnn_types           = [nn.RNN, nn.LSTM, nn.GRU]
cols                = ['Max_length', 'RNN_Type', 'Hidden_layers', 'Loss']
PATH7               = '/content/df_tt'

In [ ]:
def test_model_hw_7(max_length_sent, num_hidden_lay_list, hidden_size = 256,
                    learn_rate = 0.006, prefix = None,
                    teach_force_ratio = 0.5,
                    num_iters = 50000, every = 1000, num_file = 998):

    input_lang, output_lang, pair_s = prepareData(lang1, lang2, prefix, max_length_sent)
    print(random.choice(pair_s))

    df = pd.DataFrame(columns= cols)

    for rnn_type, num_layers in product(rnn_types, num_hidden_lay_list):

        encoder2 = EncoderRNN2(input_lang.n_words, hidden_size, num_layers, rnn_type
                               ).to(device)
        decoder2 = DecoderRNN2(hidden_size, output_lang.n_words, num_layers, rnn_type
                               ).to(device)

        print('\nMax_length = ', max_length_sent)
        print(f'{rnn_type.__name__}, {num_layers} layers\n\ntraining')
        loss_= trainIters(pair_s, encoder2, decoder2, learn_rate, num_iters,
                          max_length_sent, rnn_type, num_layers, teach_force_ratio,
                          input_lang, output_lang, every)

        print('\nevaluate\n')
        evaluateRandomly(pair_s, encoder2, decoder2, max_length_sent,
                         num_layers, rnn_type, input_lang, output_lang)

        string          = [max_length_sent, rnn_type.__name__, num_layers, loss_]
        df.loc[len(df)] = string
        file_name       = PATH7 + '.csv'
        df.to_csv(file_name)

#### 4.1.  Модель

In [ ]:
test_model_hw_7(max_length_sent = 20, num_hidden_lay_list = [1,3],
                learn_rate = 0.0001, teach_force_ratio = 0.5,
                prefix = eng_prefixes, num_iters = 20000, every = 1000,
                num_file = 1)

Reading lines...
Read 464010 sentence pairs
Trimmed to 28626 sentence pairs
Counting words...
Counted words:
rus 4274
eng 10267
['you re not going to cry are you ?', 'вы же не будете плакать ?']

Max_length =  20
RNN, 1 layers

training
3m 15s (- 61m 45s) (1000 5%) 4.7512
6m 35s (- 59m 17s) (2000 10%) 4.6375
9m 58s (- 56m 29s) (3000 15%) 4.6800
13m 23s (- 53m 35s) (4000 20%) 4.6121
16m 51s (- 50m 33s) (5000 25%) 4.6672
20m 21s (- 47m 30s) (6000 30%) 4.5790
23m 52s (- 44m 19s) (7000 35%) 4.4837
27m 23s (- 41m 4s) (8000 40%) 4.5036
30m 56s (- 37m 48s) (9000 45%) 4.4190
34m 30s (- 34m 30s) (10000 50%) 4.5056
38m 9s (- 31m 13s) (11000 55%) 4.5351
41m 48s (- 27m 52s) (12000 60%) 4.4722
45m 28s (- 24m 29s) (13000 65%) 4.4502
49m 8s (- 21m 3s) (14000 70%) 4.4890
52m 48s (- 17m 36s) (15000 75%) 4.4634
56m 26s (- 14m 6s) (16000 80%) 4.4955
60m 7s (- 10m 36s) (17000 85%) 4.4800
63m 50s (- 7m 5s) (18000 90%) 4.4157
67m 33s (- 3m 33s) (19000 95%) 4.4397
71m 18s (- 0m 0s) (20000 100%) 4.3987

evalu

In [ ]:
pd.pivot_table(pd.read_csv('/content/df_tt.csv').iloc[:,1:],
               index=["RNN_Type"],
               values=["Loss"],
               columns=["Hidden_layers"],
               aggfunc=[np.max])

amax          
                   Loss          
Hidden_layers         1         3
RNN_Type                         
GRU            2.677981  3.509464
LSTM           3.015960  3.718794
RNN            4.398707  4.745136